In [1]:
import os

from langchain_community.graphs import Neo4jGraph

os.environ["NEO4J_URI"] = "bolt://localhost:7687"
os.environ["NEO4J_USERNAME"] = "neo4j"
os.environ["NEO4J_PASSWORD"] = "2wsx3edc"

graph = Neo4jGraph()

In [2]:
from graphdatascience import GraphDataScience 

gds = GraphDataScience( 
    os.environ[ "NEO4J_URI" ], 
    auth=(os.environ[ "NEO4J_USERNAME" ], os.environ[ "NEO4J_PASSWORD" ]) 
)

In [3]:
gds.graph.drop("communities")
G, result = gds.graph.project(
    "communities",  #  Graph name
    "__Entity__",  #  Node projection
    {
        "_ALL_": {
            "type": "*",
            "orientation": "UNDIRECTED",
            "properties": {"weight": {"property": "*", "aggregation": "COUNT"}},
        }
    },
)

In [4]:
wcc = gds.wcc.stats(G)
print(f"Component count: {wcc['componentCount']}")
print(f"Component distribution: {wcc['componentDistribution']}")

Component count: 4
Component distribution: {'min': 1, 'p5': 1, 'max': 3, 'p999': 3, 'p99': 3, 'p1': 1, 'p10': 1, 'p90': 3, 'p50': 2, 'p25': 1, 'p75': 2, 'p95': 3, 'mean': 2.0}


In [5]:
gds.leiden.write(
    G,
    writeProperty="communities",
    includeIntermediateCommunities=True,
    relationshipWeightProperty="weight",
)

writeMillis                                                              5
nodePropertiesWritten                                                    8
ranLevels                                                                1
didConverge                                                           True
nodeCount                                                                8
communityCount                                                           4
communityDistribution    {'min': 1, 'p5': 1, 'max': 3, 'p999': 3, 'p99'...
modularity                                                           0.625
modularities                                                       [0.625]
postProcessingMillis                                                     3
preProcessingMillis                                                      0
computeMillis                                                           49
configuration            {'writeProperty': 'communities', 'theta': 0.01...
Name: 0, dtype: object

In [6]:
graph.query("""
MATCH (e:`__Entity__`)
UNWIND range(0, size(e.communities) - 1 , 1) AS index
CALL {
  WITH e, index
  WITH e, index
  WHERE index = 0
  MERGE (c:`__Community__` {id: toString(index) + '-' + toString(e.communities[index])})
  ON CREATE SET c.level = index
  MERGE (e)-[:IN_COMMUNITY]->(c)
  RETURN count(*) AS count_0
}
CALL {
  WITH e, index
  WITH e, index
  WHERE index > 0
  MERGE (current:`__Community__` {id: toString(index) + '-' + toString(e.communities[index])})
  ON CREATE SET current.level = index
  MERGE (previous:`__Community__` {id: toString(index - 1) + '-' + toString(e.communities[index - 1])})
  ON CREATE SET previous.level = index - 1
  MERGE (previous)-[:IN_COMMUNITY]->(current)
  RETURN count(*) AS count_1
}
RETURN count(*)
""")

[{'count(*)': 8}]

In [7]:
graph.query("""
MATCH (c:__Community__)<-[:IN_COMMUNITY*]-(:__Entity__)<-[:HAS_ENTITY]-(d:__Chunk__)
WITH c, count(distinct d) AS rank
SET c.rank = rank;
""")

[]

In [8]:
import pandas as pd
import numpy as np

community_size = graph.query(
    """
MATCH (c:__Community__)<-[:IN_COMMUNITY*]-(e:__Entity__)
WITH c, count(distinct e) AS entities
RETURN split(c.id, '-')[0] AS level, entities
"""
)
community_size_df = pd.DataFrame.from_records(community_size)
percentiles_data = []
for level in community_size_df["level"].unique():
    subset = community_size_df[community_size_df["level"] == level]["entities"]
    num_communities = len(subset)
    percentiles = np.percentile(subset, [25, 50, 75, 90, 99])
    percentiles_data.append(
        [
            level,
            num_communities,
            percentiles[0],
            percentiles[1],
            percentiles[2],
            percentiles[3],
            percentiles[4],
            max(subset)
        ]
    )

# Create a DataFrame with the percentiles
percentiles_df = pd.DataFrame(
    percentiles_data,
    columns=[
        "Level",
        "Number of communities",
        "25th Percentile",
        "50th Percentile",
        "75th Percentile",
        "90th Percentile",
        "99th Percentile",
        "Max"
    ],
)
percentiles_df

,Level,Number of communities,25th Percentile,50th Percentile,75th Percentile,90th Percentile,99th Percentile,Max
0,0,4,1.75,2.0,2.25,2.7,2.97,3


In [9]:
community_info = graph.query("""
MATCH (c:`__Community__`)<-[:IN_COMMUNITY*]-(e:__Entity__)
WHERE c.level IN [0,1,4]
WITH c, collect(e ) AS nodes
WHERE size(nodes) > 1
CALL apoc.path.subgraphAll(nodes[0], {
 whitelistNodes:nodes
})
YIELD relationships
RETURN c.id AS communityId, 
       [n in nodes | {id: n.id, description: n.description, type: [el in labels(n) WHERE el <> '__Entity__'][0]}] AS nodes,
       [r in relationships | {start: startNode(r).id, type: type(r), end: endNode(r).id, description: r.description}] AS rels
""")
community_info

[{'communityId': '0-6',
  'nodes': [{'id': '活期帳戶扣款',
    'description': '从客户的活期账户中自动扣除费用',
    'type': 'Accountdeduction'},
   {'id': '保戶', 'description': '购买保险并持有保险合同的人', 'type': 'Policyholder'}],
  'rels': [{'start': '保戶',
    'description': None,
    'type': 'DEDUCTS_FROM',
    'end': '活期帳戶扣款'}]},
 {'communityId': '0-3',
  'nodes': [{'id': '新契約投保時授權書',
    'description': '不得為空白授權，應依是否取得保單號碼填寫相關欄位',
    'type': 'Document'},
   {'id': '自動轉帳及信用卡付款授權書作業規範', 'description': '加強身分驗證機制', 'type': 'Rule'}],
  'rels': [{'start': '新契約投保時授權書',
    'description': None,
    'type': 'REFER_TO',
    'end': '自動轉帳及信用卡付款授權書作業規範'}]},
 {'communityId': '0-5',
  'nodes': [{'id': '首期保險費',
    'description': '首次需繳交的保險費用',
    'type': 'Insurancefee'},
   {'id': '金融機構轉帳作業',
    'description': '通過銀行或金融机构進行转账支付的作业方式',
    'type': 'Paymentmethod'},
   {'id': '本公司', 'description': '提供保险服务的公司', 'type': 'Insurancecompany'}],
  'rels': [{'start': '首期保險費',
    'description': None,
    'type': 'PAID_BY',
    'end': '金融

In [10]:
# import os

# from langchain_experimental.graph_transformers import LLMGraphTransformer
# # from langchain_openai import ChatOpenAI
# from langchain_openai import AzureChatOpenAI

# llm = AzureChatOpenAI(
#     azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
#     azure_deployment=os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"],
#     openai_api_version=os.environ["AZURE_OPENAI_API_VERSION"],
# )

In [10]:
from langchain_experimental.llms.ollama_functions import OllamaFunctions
model_name = 'qwen2:72b-instruct-q8_0'
llm = OllamaFunctions(model=model_name)
llm

/storage/pyenv/versions/3.12.5/envs/ACP_LLM_312/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:141: LangChainDeprecationWarning: The class `OllamaFunctions` was deprecated in LangChain 0.0.64 and will be removed in 0.4.0. An updated version of the class exists in the langchain-ollama package and should be used instead. To use it run `pip install -U langchain-ollama` and import as `from langchain_ollama import ChatOllama`.
  warn_deprecated(


OllamaFunctions(model='qwen2:72b-instruct-q8_0')

In [13]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

community_template = """請根據提供同社區的資訊包含 nodes 與 relationships,
產生自然語言的摘要 同社區的資訊
請確保使用繁體中文回應:
{community_info}

Summary:"""  # noqa: E501

community_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Given an input triples, generate the information summary. No pre-amble.",
        ),
        ("human", community_template),
    ]
)

community_chain = community_prompt | llm | StrOutputParser()

In [14]:
def prepare_string(data):
    nodes_str = "Nodes are:\n"
    for node in data['nodes']:
        node_id = node['id']
        node_type = node['type']
        if 'description' in node and node['description']:
            node_description = f", description: {node['description']}"
        else:
            node_description = ""
        nodes_str += f"id: {node_id}, type: {node_type}{node_description}\n"

    rels_str = "Relationships are:\n"
    for rel in data['rels']:
        start = rel['start']
        end = rel['end']
        rel_type = rel['type']
        if 'description' in rel and rel['description']:
            description = f", description: {rel['description']}"
        else:
            description = ""
        rels_str += f"({start})-[:{rel_type}]->({end}){description}\n"

    return nodes_str + "\n" + rels_str

def process_community(community):
    stringify_info = prepare_string(community)
    summary = community_chain.invoke({'community_info': stringify_info})
    return {"community": community['communityId'], "summary": summary}

process_community(community_info[0])

{'community': '0-6',
 'summary': '在這個社區中，資訊顯示保戶會從他們的活期帳戶中自動扣除費用，這表示保單持有人購買了保險並且持有合約，而相關的費用則是直接從他們的活期帳戶中進行扣款。'}

In [15]:
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm

summaries = []
with ThreadPoolExecutor() as executor:
    futures = {executor.submit(process_community, community): community for community in community_info}

    for future in tqdm(as_completed(futures), total=len(futures), desc="Processing communities"):
        summaries.append(future.result())

Processing communities: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:34<00:00, 11.47s/it]


In [16]:
summaries

[{'community': '0-3',
  'summary': '在同社區的資訊中，有一份名為"新契約投保時授權書"的文件，該文件要求不得為空白授權，並應根據是否取得保單號碼填寫相關欄位。此外，此文件參照了"自動轉帳及信用卡付款授權書作業規範"的規定，強調在進行轉帳或信用卡付款前，需要加強身分驗證機制。'},
 {'community': '0-6', 'summary': '根據提供的資訊，保單持有人會從他們的活期賬戶中自動扣除相關費用。'},
 {'community': '0-5',
  'summary': '此同社區的資訊描述了一家提供保險服務的公司，該公司在付款方式上使用了通過銀行或金融機構進行的轉賬支付。首期的保險費用是通過這種金融機構轉帳作業方式來繳交的。'}]

In [17]:
graph.query("""
UNWIND $data AS row
MERGE (c:__Community__ {id:row.community})
SET c.summary = row.summary
""", params={"data": summaries})

[]